In speech processing we may use really huge networks cause majority of the processing bottleneck is on CPU and running the CTC function...

Also, CLARIN mobile corpus seems to be too small for advanced network

In [2]:
import tensorflow as tf
import numpy as np
import keras.backend as K
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

STDS = K.random_uniform([4, 4])
MEANS = K.random_uniform([4])
x = K.random_uniform([4])

with tf.Session() as sess:
    a = tf.sqrt(1. / tf.linalg.det(2 * np.pi * STDS)).eval(session=sess)
    b = tf.exp(-0.5 * tf.transpose(K.expand_dims((x - MEANS))) @ tf.linalg.inv(STDS) @ K.expand_dims((x - MEANS))).eval(session=sess)

print(a * b)

/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[[0.21273977]]


Using TensorFlow backend.


In [31]:
import keras
from keras import activations, initializers, regularizers, constraints
from keras.layers import InputSpec

def covinit(shape, dtype=None):
    return 0.1 * K.random_normal(shape, dtype=dtype) + K.tile(K.expand_dims(K.eye(shape[1], dtype=dtype), axis=0), [shape[0], 1, 1])

class GaussianLayer(keras.layers.Layer):
    def __init__(self, units,
                 activation=None,
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(GaussianLayer, self).__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.input_spec = InputSpec(min_ndim=2)
        self.supports_masking = True

    def build(self, input_shape):
        assert len(input_shape) >= 2
        self.input_dim = input_dim = input_shape[-1]
        self.means =  self.add_weight(shape=(self.units, input_dim),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        self.std =  self.add_weight(shape=(self.units, input_dim, input_dim),
                                      initializer=covinit,
                                      name='kernelstd',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

    def call(self, inputs):
        def gaussian_op(mean, std, inputs):
            time = inputs.shape[0]
            time = time if time is not None else -1
            std_in_time = K.tile(K.expand_dims(std, axis=0), [time, 1, 1, 1])
            std_inv = K.tile(K.expand_dims(tf.linalg.inv(std), axis=0), [time, 1, 1, 1])
            o_feats = std.shape[0]
            x = K.tile(K.expand_dims(inputs, axis=1), [1, o_feats, 1])
            a = tf.sqrt(1. / tf.linalg.det(2 * np.pi * std_in_time))
            b = -0.5 * K.expand_dims((x - mean), axis=2) @ std_inv @ K.expand_dims((x - mean), axis=3)
            a = tf.log(a)
            result = K.expand_dims(K.expand_dims(a, axis=2), axis=3) + b
            result = K.sum(result, axis=(2,3))
            return result
        std = self.std @ tf.transpose(self.std, perm=[0, 2, 1]) + 1e-12 * tf.tile(K.expand_dims(K.eye(self.input_dim), axis=0), [self.units, 1, 1])
        fn = lambda x: gaussian_op(self.means, std, x)
        result = tf.map_fn(fn, inputs)
        return result

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

    def get_config(self):
        config = {
            'units': self.units,
            'activation': activations.serialize(self.activation),
            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'activity_regularizer': regularizers.serialize(self.activity_regularizer),
            'kernel_constraint': constraints.serialize(self.kernel_constraint)
        }
        base_config = super(GaussianLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [32]:
import keras
from keras.layers import Dense

import numpy as np

m = keras.models.Sequential()
m.add(GaussianLayer(32, input_shape=(128, 20)))
m.add(BatchNormalization())
m.add(GaussianLayer(16))
m.add(BatchNormalization())
m.add(Dense(4))
m.compile('adam', 'mse')
m.fit(np.random.random([6400, 128, 20]), np.random.random([6400, 128, 4]))

Epoch 1/1
 192/6400 [..............................] - ETA: 3:46 - loss: 1.6852

KeyboardInterrupt: 

In [23]:
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_layer_13 (GaussianL (None, 128, 32)           13440     
_________________________________________________________________
batch_normalization_10 (Batc (None, 128, 32)           128       
_________________________________________________________________
gaussian_layer_14 (GaussianL (None, 128, 16)           16896     
_________________________________________________________________
batch_normalization_11 (Batc (None, 128, 16)           64        
_________________________________________________________________
dense_3 (Dense)              (None, 128, 4)            68        
Total params: 30,596
Trainable params: 30,500
Non-trainable params: 96
_________________________________________________________________


In [13]:
#!/usr/bin/python3

# For demonstration purposes - Paweł Tomasik

# for CLARIN_MOBILE - generally it is unnormalized

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from keras.models import Model
from keras.layers import LSTM, Conv1D, Dropout, LeakyReLU, Dense, Input, Lambda, TimeDistributed, Flatten, Conv2D, BatchNormalization, GRU
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import Orthogonal

from keras.backend import ctc_batch_cost, expand_dims
import keras.backend as K

import editdistance  # For digit error rate
import keras
import librosa
import numpy as np
import os

LENGTH = 1109
RECS = 1384
TRANSL = 307

X = np.zeros([RECS, LENGTH, 20], np.float32)
Y = np.zeros([RECS, TRANSL], np.int16)
counter = 0

for i in range(1):
    Xpart = np.load("datasets/clarin-long/data/clarin-mfcc-rec-{}.npy".format(i))
    Ypart = np.load("datasets/clarin-long/data/clarin-mfcc-trans-{}.npy".format(i))
    recs = Xpart.shape[0]
    reclen = Xpart.shape[1]
    translen = Ypart.shape[1]
    X[counter : counter + recs, :reclen, :] = Xpart
    Y[counter : counter + recs, :translen] = Ypart
    counter += recs


X = X[:4]
Y = Y[:4]

#X = np.clip(X, -3, 3)
MEAN = X.mean()
STD = X.std()


In [33]:
def mk_model(max_label_length):
    feature_input = Input(shape = (1109, NFEATS))
    gaussians = GaussianLayer(64)(feature_input)
    norm = BatchNormalization()(gaussians)
    gaussians = GaussianLayer(64)(norm)
    norm = BatchNormalization()(gaussians)
    final = LSTM(NPHONES + 1, return_sequences = True, activation = 'softmax')(norm)
    label_input = Input(shape = (max_label_length,))
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    loss_lambda = Lambda(lambda args:ctc_batch_cost(*args), output_shape=(1,), name='ctc')([label_input, final, input_length, label_length])
    model = Model([feature_input, label_input, input_length, label_length], [loss_lambda])
    model.summary()
    predictive = Model(feature_input, final)
    return model, predictive

def train(model, trainX, trainy, trainXl, trainyl, epochs = 50):
    # important: batch_size=1 bugs Tensorflow
    optimizer = Adam(0.0003, clipnorm=1.)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    return model.fit([trainX, trainy, trainXl, trainyl], np.zeros(trainX.shape[0]), epochs = epochs, batch_size = 2)

def validate(predictions, valid_length, groundtruth, target_length):
    predictions = keras.backend.ctc_decode(predictions, valid_length, False, 1)
    predictions = predictions[0][0].eval(session=keras.backend.get_session())
    DERs = []
    for index in range(predictions.shape[0]):
        dist = float(editdistance.eval(
            [x for x in predictions[index, :] if x != -1],
            [x for x in groundtruth[index, :] if x != NPHONES]))
        DER = dist / target_length[index]
        DERs.append((DER, target_length[index]))
    return DERs

def try_else(exp, exp_else):
    try:
        return exp()
    except:
        return exp_else

if __name__=='__main__':
    data = X, Y
    NPHONES = Y.max()
    NFEATS = data[0].shape[2]
    X_lens = np.array([try_else(
            (lambda:np.where((x).mean(1) == (x).std(1))[0][0]),
            X.shape[1])
        for x in X])
    X_lens = np.ceil(X_lens)
    Y_lens = np.array([np.where(x == NPHONES)[0] for x in data[1]])
    Y_lens = np.array([x[0] if len(x) else 0 for x in Y_lens])
    data = data[0][np.where(Y_lens)], data[1][np.where(Y_lens)], X_lens[np.where(Y_lens)].reshape(-1, 1), Y_lens[np.where(Y_lens)].reshape(-1, 1)
    print(list(map(lambda x:x.shape, data)))
    trn, predict = mk_model(data[1].shape[1])
    train(trn, *data, epochs=1000) # at 300 it makes sensible predictions
    valid_data = make_data(*load_data(VALIDDATA))
    predictions = predict.predict(valid_data[0])
    DERs = validate(predictions, valid_data[2], valid_data[1], valid_data[3])
    print("Validation Digit Error Rate: {}".format(sum([x[0] * x[1] for x in DERs]) / sum([x[1] for x in DERs])))
    predict.save('cyfry.pred.h5')

[(4, 1109, 20), (4, 307), (4, 1), (4, 1)]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 1109, 20)     0                                            
__________________________________________________________________________________________________
gaussian_layer_25 (GaussianLaye (None, 1109, 64)     26880       input_17[0][0]                   
__________________________________________________________________________________________________
batch_normalization_21 (BatchNo (None, 1109, 64)     256         gaussian_layer_25[0][0]          
__________________________________________________________________________________________________
gaussian_layer_26 (GaussianLaye (None, 1109, 64)     266240      batch_normalization_21[0][0]     
___________________________________________________________________

InvalidArgumentError: Input is not invertible.
	 [[node gaussian_layer_25/map/while/MatrixInverse (defined at <ipython-input-31-ef4aa1c483e5>:49)  = MatrixInverse[T=DT_FLOAT, _class=["loc:@train...e_grad/Neg"], adjoint=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](gaussian_layer_25/map/while/ExpandDims/Enter, ^gaussian_layer_25/map/while/Identity)]]

Caused by op 'gaussian_layer_25/map/while/MatrixInverse', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/venv/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/venv/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/venv/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/venv/lib/python3.

In [18]:
tf.linalg.inv(trn.layers[1].std)

<tf.Tensor 'MatrixInverse_1:0' shape=(64, 20, 20) dtype=float32>

In [19]:
tf.linalg.inv(trn.layers[3].std)

<tf.Tensor 'MatrixInverse_2:0' shape=(64, 64, 64) dtype=float32>